In [1]:
import pandas as pd

In [2]:
# Read cleaned dataframes
census_2011 = pd.read_csv("/home/matthewdshen/GitHub/urban_data_project/analysis/cleaned_features/census_2011.csv", low_memory=False)
census_2012 = pd.read_csv("/home/matthewdshen/GitHub/urban_data_project/analysis/cleaned_features/census_2012.csv", low_memory=False)
census_2013 = pd.read_csv("/home/matthewdshen/GitHub/urban_data_project/analysis/cleaned_features/census_2013.csv", low_memory=False)
census_2014 = pd.read_csv("/home/matthewdshen/GitHub/urban_data_project/analysis/cleaned_features/census_2014.csv", low_memory=False)
census_2015 = pd.read_csv("/home/matthewdshen/GitHub/urban_data_project/analysis/cleaned_features/census_2015.csv", low_memory=False)
census_2016 = pd.read_csv("/home/matthewdshen/GitHub/urban_data_project/analysis/cleaned_features/census_2016.csv", low_memory=False)
census_2017 = pd.read_csv("/home/matthewdshen/GitHub/urban_data_project/analysis/cleaned_features/census_2017.csv", low_memory=False)
census_2018 = pd.read_csv("/home/matthewdshen/GitHub/urban_data_project/analysis/cleaned_features/census_2018.csv", low_memory=False)
census_2019 = pd.read_csv("/home/matthewdshen/GitHub/urban_data_project/analysis/cleaned_features/census_2019.csv", low_memory=False)

In [3]:
# Group data based on zipcode
census_2011 = census_2011.groupby(['NAME'], as_index=False).mean()
census_2012 = census_2012.groupby(['NAME'], as_index=False).mean()
census_2013 = census_2013.groupby(['NAME'], as_index=False).mean()
census_2014 = census_2014.groupby(['NAME'], as_index=False).mean()
census_2015 = census_2015.groupby(['NAME'], as_index=False).mean()
census_2016 = census_2016.groupby(['NAME'], as_index=False).mean()
census_2017 = census_2017.groupby(['NAME'], as_index=False).mean()
census_2018 = census_2018.groupby(['NAME'], as_index=False).mean()
census_2019 = census_2019.groupby(['NAME'], as_index=False).mean()

In [4]:
# Interpolate data to fill in missing values
census_2011.interpolate(inplace = True)
census_2012.interpolate(inplace = True)
census_2013.interpolate(inplace = True)
census_2014.interpolate(inplace = True)
census_2015.interpolate(inplace = True)
census_2016.interpolate(inplace = True)
census_2017.interpolate(inplace = True)
census_2018.interpolate(inplace = True)
census_2019.interpolate(inplace = True)

In [5]:
# Fill first value if na
census_2011.fillna(method = 'bfill', inplace = True)
census_2012.fillna(method = 'bfill', inplace = True)
census_2013.fillna(method = 'bfill', inplace = True)
census_2014.fillna(method = 'bfill', inplace = True)
census_2015.fillna(method = 'bfill', inplace = True)
census_2016.fillna(method = 'bfill', inplace = True)
census_2017.fillna(method = 'bfill', inplace = True)
census_2018.fillna(method = 'bfill', inplace = True)
census_2019.fillna(method = 'bfill', inplace = True)


In [6]:
# Get difference in values from 2011 and 2019
df_2011_2019 = pd.merge(census_2011,census_2019,how = 'inner', on = 'NAME', suffixes=['_2011','_2019'])

# Get list of values
lst_values = ['NAME','median_age','white_non-hispanic','household_income','foreign_born_not_a_us_citizen','bachelors','gross_rent_media_price']

# Intialize dataframe
df_delta = pd.DataFrame(df_2011_2019['NAME'])

# Go through each element of the list values
for feature in lst_values[1:]:
    df_delta[feature + '_delta'] = df_2011_2019[feature + '_2019'] - df_2011_2019[feature + '_2011'] # get the change in census tracts

# Switch signs so that gentrification can be seen as positive
df_delta['median_age_delta'] = df_delta['median_age_delta'] * -1
df_delta['foreign_born_not_a_us_citizen_delta'] = df_delta['foreign_born_not_a_us_citizen_delta'] * -1

In [7]:
# Normalize the data
def normalize(df):
    '''The following normalizes the data'''
    result = df.copy()
    for feature_name in df.columns[1:]:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result
df_delta_norm = normalize(df_delta)

# Get the mean of the normalized data
df_mean = df_delta_norm.mean()

# Get sum of mean values
int_gentrification_metric_mean = sum(df_mean)

/tmp/ipykernel_18394/248879814.py:13: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_mean = df_delta_norm.mean()


In [11]:
# Get getrification metric for each ZCTA5
df_delta_norm['gentrification_metric'] = df_delta_norm.sum( axis = 'columns')

# Get difference between city wide gentrification metric and individual zipcode
df_delta_norm['gentrification_metric_delta'] = df_delta_norm['gentrification_metric'] - int_gentrification_metric_mean


df_delta_norm[df_delta_norm['NAME'] == 'ZCTA5 11249']

/tmp/ipykernel_18394/1845456440.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_delta_norm['gentrification_metric'] = df_delta_norm.sum( axis = 'columns')


,NAME,median_age_delta,white_non-hispanic_delta,household_income_delta,foreign_born_not_a_us_citizen_delta,bachelors_delta,gross_rent_media_price_delta,gentrification_metric,gentrification_metric_delta
0,ZCTA5 11249,0.474087,0.423861,0.218294,0.992403,0.937267,0.479367,13.742723,10.097980
1,ZCTA5 11249,0.423959,0.498112,0.636243,0.855008,0.669252,0.464114,13.892591,10.247849
2,ZCTA5 11249,0.404418,0.374679,0.561957,0.475141,0.465280,0.358903,7.548416,3.903673
3,ZCTA5 11249,0.443500,0.310194,0.652627,0.831114,0.138620,0.475320,9.025402,5.380660
4,ZCTA5 11249,0.406967,0.445255,0.773164,0.980519,0.892458,0.810877,19.230455,15.585712
...,...,...,...,...,...,...,...,...,...
1789,ZCTA5 11249,0.423110,0.418424,0.579500,0.992278,0.945286,0.316258,14.789758,11.145015
1790,ZCTA5 11249,0.437553,0.390435,0.565387,0.986160,0.955715,0.336802,14.770140,11.125397
1791,ZCTA5 11249,0.401869,0.452253,0.610277,0.987865,0.936324,0.323417,15.049808,11.405066
1792,ZCTA5 11249,0.446899,0.357312,0.582766,0.986737,0.966773,0.323728,14.715277,11.070534
